In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from sensitivity_tests import *

import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
import os
import numpy as np
import math
import random
import sys
import itertools
from scipy import stats
import pandas as pd
sys.path.append("~/rankability_toolbox")
import pyrankability

from gurobipy import *
setParam("OutputFlag", 0)
setParam("Presolve", 0)

In [ ]:
# Produces a pandas Series with the optimal rankings found from each permutation
def solve_all_permutations(D_orig):
    perms = list(itertools.permutations(range(D_orig.shape[0])))
    P_elems = []
    for perm in tqdm(perms):
        D_permuted = D_orig[perm,:][:,perm]
        k, details = pyrankability.lop.bilp(D_permuted)
        elem = list(details["P"][0])
        for i in range(len(elem)):
            elem[i] = perm[elem[i]]
        P_elems.append(str(elem))
    return pd.Series(P_elems)

# Checks if the frequencies are uniform or not
def check_freqs(freqs):
    if len(freqs.unique()) > 1:
        print(freqs)
        plt.bar(range(1, len(freqs)+1), height=freqs.values)
        plt.show()
        return False
    else:
        print("Perfectly balanced, as all things should be.")
        print("All {0} elements produced by {1} permutations".format(len(freqs), freqs.unique()[0]), flush=True)
        return True  

# Generates random noisy D matrices and checks for uniform frequency
def test_rand_D(noise_gen, num_attempts):
    for i in range(num_attempts):
        D2 = noise_gen.apply_noise(D)
        print(D2)
        P_elems_series = solve_all_permutations(D2)
        freqs = P_elems_series.value_counts()
        check_freqs(freqs)

def test_rand_D_correlate_sensitivity(noise_gen, num_attempts):
    for i in range(num_attempts):
        D2 = noise_gen.apply_noise(D)
        print(D2)
        P_elems_series = solve_all_permutations(D2)
        freqs = P_elems_series.value_counts()
        if not check_freqs(freqs):
            D2_P = pd.Series(freqs.index.values)
            D2_P_counts = D2_P.copy(deep=True)
            D2_P_counts.loc[:] = 0
            noise_gen2 = BinaryFlipNoise(0.05)
            for k in range(16):
                D3 = noise_gen2.apply_noise(D2)
                D3_P = solve_all_permutations(D3).unique()
                D2_P_counts = D2_P_counts.add(D2_P.isin(D3_P).astype(int))
            plt.bar(range(1, len(D2_P_counts)+1), height=D2_P_counts.values)
            plt.show()

## Perfect Dominance with Constant # of Ties

In [ ]:
n_items = 7
n_ties = 4

# Create perfect binary dominance
D = PerfectBinarySource(n_items).init_D()

# Tie-up the bottom n_ties items
for i in range(n_ties):
    D[n_items-i-1] = 0

D

In [ ]:
n_ties_series = solve_all_permutations(D)
freqs = n_ties_series.value_counts()
check_freqs(freqs)

## Re-initialize D

In [ ]:
D = PerfectBinarySource(n_items).init_D()
D

## Swap Noise

In [ ]:
test_rand_D(SwapNoise(0.5), 4)

## Binary Flip Noise

In [ ]:
test_rand_D(BinaryFlipNoise(0.5), 4)

## Normal Resampling Noise

In [ ]:
test_rand_D(NormalResamplingNoise(0.5), 4)